<h2 align='center'> ML Flow </h2>

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [5]:
import numpy as np

np.unique(y_train_res, return_counts=True)


(array([0, 1]), array([619, 619]))

In [7]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)


(array([0, 1]), array([619, 619]))

### Track Experiments

In [8]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [9]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [10]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [13]:
# Initialize MLflow
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_experiment("Anomaly Detection")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/04/09 16:11:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/189956448505815174/runs/a5397bf2dd1c47c4bd7353a4acdb26ac
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/189956448505815174


2025/04/09 16:11:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/189956448505815174/runs/98d628ded1ed46968b3462cafe4ad470
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/189956448505815174


2025/04/09 16:11:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/189956448505815174/runs/bc4f1e3943984848ac47f2f1573ea8ea
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/189956448505815174


2025/04/09 16:11:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/189956448505815174/runs/94c431a0bab14e0ca46cbb3a0c886c28
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/189956448505815174


### Register the Model

In [16]:
import mlflow

# Define your run ID and model name
run_id = "94c431a0bab14e0ca46cbb3a0c886c28"
model_name = "MyAnomalyDetectionModel"  # You can customize this name

# Make sure this matches the artifact_path used in mlflow.log_model(..., artifact_path="model")
model_uri = f"runs:/{run_id}/model"

# Register the model
mlflow.register_model(model_uri=model_uri, name=model_name)


Successfully registered model 'MyAnomalyDetectionModel'.
2025/04/09 16:13:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: MyAnomalyDetectionModel, version 1
Created version '1' of model 'MyAnomalyDetectionModel'.


<ModelVersion: aliases=[], creation_timestamp=1744195429681, current_stage='None', description='', last_updated_timestamp=1744195429681, name='MyAnomalyDetectionModel', run_id='94c431a0bab14e0ca46cbb3a0c886c28', run_link='', source='mlflow-artifacts:/189956448505815174/94c431a0bab14e0ca46cbb3a0c886c28/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='1'>

### Load the Model

In [17]:
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Transition the Model to Production

In [ ]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

In [19]:
mlflow.register_model(
    model_uri="runs:/94c431a0bab14e0ca46cbb3a0c886c28/model",
    name="MyAnomalyDetectionModel"
)


Registered model 'MyAnomalyDetectionModel' already exists. Creating a new version of this model...
2025/04/09 16:14:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: MyAnomalyDetectionModel, version 2
Created version '2' of model 'MyAnomalyDetectionModel'.


<ModelVersion: aliases=[], creation_timestamp=1744195490426, current_stage='None', description='', last_updated_timestamp=1744195490426, name='MyAnomalyDetectionModel', run_id='94c431a0bab14e0ca46cbb3a0c886c28', run_link='', source='mlflow-artifacts:/189956448505815174/94c431a0bab14e0ca46cbb3a0c886c28/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='2'>

In [ ]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 

In [ ]:
#Complete workflow


import mlflow
import mlflow.xgboost
from mlflow.models.signature import infer_signature
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pandas as pd
from mlflow.tracking import MlflowClient

# Load and split data
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

# MLflow config
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_experiment("Anomaly Detection")

# Start and log run
with mlflow.start_run(run_name="XGBoostAnomalyModel") as run:
    run_id = run.info.run_id
    
    # Log model
    input_example = X_test.head(2)
    signature = infer_signature(X_test, model.predict(X_test))
    
    mlflow.xgboost.log_model(
        model,
        artifact_path="model",
        input_example=input_example,
        signature=signature
    )

    model_uri = f"runs:/{run_id}/model"
    registered_model_name = "MyAnomalyDetectionModel"

    # Register model
    mlflow.register_model(
        model_uri=model_uri,
        name=registered_model_name
    )
    
    print("✅ Model logged and registered.")


Registered model 'MyAnomalyDetectionModel' already exists. Creating a new version of this model...
2025/04/09 16:17:18 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: MyAnomalyDetectionModel, version 3


✅ Model logged and registered.
🏃 View run XGBoostAnomalyModel at: http://127.0.0.1:5000/#/experiments/189956448505815174/runs/ced380125a904952a3b56daed6936081
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/189956448505815174


Created version '3' of model 'MyAnomalyDetectionModel'.
